In [36]:
import os
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K

In [2]:
%matplotlib inline


tf.enable_resource_variables()


PARAMETERS

The values used in the base model of transformer were; num_layers=6, d_model = 512, dff = 2048. See the paper for all the other versions of the transformer.


In [3]:
BERT_MODEL_URL = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

SEQ_LEN = 512
MAX_EXAMPLE_LEN = 512

BATCH_SIZE = 1
BUFFER_SIZE = 1

num_layers = 4
d_model = 128
dff = 512
num_heads = 8

# input_vocab_size = tokenizer_pt.vocab_size + 2
# target_vocab_size = tokenizer_en.vocab_size + 2

dropout_rate = 0.1

In [6]:
from data.load import load_cnn_dailymail
from ops.session import initialize_vars


train_dataset, val_dataset, test_dataset, metadata = load_cnn_dailymail()  

Instructions for updating:
Colocations handled automatically by placer.


W0527 20:53:04.776886 139771495298880 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0527 20:53:07.114221 139771495298880 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [7]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [8]:
from models.transformer import Transformer

In [9]:
with open(b'/tmp/tfhub_modules/5a395eafef2a37bd9fc55d7f6ae676d2a134a838/assets/vocab.txt') as f:
    data = f.readlines()


In [10]:
data[:10]

['[PAD]\n',
 '[unused0]\n',
 '[unused1]\n',
 '[unused2]\n',
 '[unused3]\n',
 '[unused4]\n',
 '[unused5]\n',
 '[unused6]\n',
 '[unused7]\n',
 '[unused8]\n']

In [29]:
j = 0
for i, line in enumerate(data):
    if 'unused' in line:
        j += 1
len(data) - j

29527

In [11]:
from bert.tokenization import FullTokenizer


BERT_MODEL_URL = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"


def create_bert_tokenizer(vocab_file, do_lower_case=True):
    """
    Return a BERT FullTokenizer
    """                
    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)


def create_tokenizer_from_hub_module(bert_hub_url, bert_module=None):
    """
    Get the vocab file and casing info from the Hub module.
    """
    if bert_module is None:
        bert_module = hub.Module(bert_hub_url)
        
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)

    with tf.Session() as sess:
        vocab_file, do_lower_case = sess.run([
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"]
        ])
        print(vocab_file)
                
    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)


tokenizer = create_tokenizer_from_hub_module(BERT_MODEL_URL)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0527 20:53:16.411491 139771495298880 saver.py:1489] Saver not created because there are no variables in the graph to restore


b'/tmp/tfhub_modules/5a395eafef2a37bd9fc55d7f6ae676d2a134a838/assets/vocab.txt'


In [12]:
bert = hub.Module(
            BERT_MODEL_URL,
            trainable=False,
            name="{}_bert_module".format('self.name')
        )

In [37]:
from layers.bert import BertLayer


class BertLayer(tf.keras.layers.Layer):
    """
    Custom Keras layer, integrating BERT from tf-hub
    """
    def __init__(self, url, seq_len=512, d_embedding=768, n_fine_tune_layers=None, **kwargs):
        self.url = url
        self.n_fine_tune_layers = n_fine_tune_layers
        self.seq_len = seq_len
        self.d_embedding = d_embedding
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.url,
            trainable=self.trainable,
            name="{}_bert_module".format(self.name)
        )

        trainable_vars = self.bert.variables
        
        if self.trainable:

            # Remove unused layers
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)

            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        
        input_ids, input_mask, segment_ids = inputs
        
        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["sequence_output"]
        return result

    def compute_output_shape(self, input_shape):
        raise ValueError()
        return (input_shape[0], self.seq_len, self.d_embedding)

    
    
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0527 20:38:47.230920 140013419611968 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [14]:
atmp1 = np.zeros((30522))
atmp1[10] = 1

atmp2 = np.zeros((30522))
atmp2[20] = 1

In [49]:
atmp = np.array([atmp1, atmp2], dtype=np.float32)
atmp.dtype

dtype('float32')

In [50]:
from tensorflow.python.keras.initializers import Constant


with tf.Session() as sess:
    
    initialize_vars(sess)
    embedding_matrix = sess.run(bert.variable_map['bert/embeddings/word_embeddings'])

target_vocab_size = len(data)
d_model = 768

embedding = tf.keras.layers.Embedding(target_vocab_size, d_model, trainable=False, embeddings_initializer=Constant(embedding_matrix))



In [51]:
embedding_matrix.shape

(30522, 768)

In [52]:
with tf.Session() as sess:
    
    initialize_vars(sess) 
    atmp_ = tf.convert_to_tensor(atmp, np.float32)
    print(atmp_)
    print(sess.run(embedding(atmp_)).shape)
    raise ValueError
    print(sess.run(tf.matmul(atmp, bert.variable_map['bert/embeddings/word_embeddings'])))
    print(sess.run(bert.variable_map['bert/embeddings/word_embeddings'](np.array([0, 1]))))


    

Tensor("Const_20:0", shape=(2, 30522), dtype=float32)


NotFoundError: Resource localhost/embedding_5/embeddings/N10tensorflow3VarE does not exist.
	 [[node embedding_5/embedding_lookup (defined at <ipython-input-52-fac9b5b8d76d>:6) ]]

Caused by op 'embedding_5/embedding_lookup', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/ec2-user/anaconda3

In [53]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    initialize_vars(sess)    
    tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = tmp
    print(x0.shape, x1.shape, x2.shape)
#     print(x0, '\n', x1)
    tmp2 = model.predict([x0, x1, x2])
    print(tmp2)
#     print(sess.run(tmp2))  # (batch_size, input_seq_len, d_model)    


    

(1, 512) (1, 512) (1, 512)


NameError: name 'model' is not defined

In [9]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:

#     sess.run(iter.initializer)
#     tmp = sess.run(el)
#     print(tmp)
    sample_transformer = Transformer(
        num_layers=2, d_model=512, num_heads=8, dff=2048, 
        input_vocab_size=8500, target_vocab_size=8000)

    temp_input = tf.random.uniform((64, 62))
    temp_target = tf.random.uniform((64, 26))

    fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                                   enc_padding_mask=None, 
                                   look_ahead_mask=None,
                                   dec_padding_mask=None)
    initialize_vars(sess)
    
    print(sess.run(fn_out))  # (batch_size, input_seq_len, d_model)    
#     x0, x1, x2, y0, y1, y2 = tmp
#     tmp2 = model.predict([x0, x1, x2])
    

[[[ 0.49737298  0.5550515  -0.13341121 ... -0.5284789  -0.61509997
   -0.04155324]
  [ 0.5157517   0.5635947  -0.09799525 ... -0.5260042  -0.62772596
   -0.03477238]
  [ 0.53440964  0.5700865  -0.05099516 ... -0.5282975  -0.63185966
   -0.02259888]
  ...
  [ 0.39933348  0.4695823  -0.19357261 ... -0.49450147 -0.70106435
   -0.01530197]
  [ 0.41170758  0.47419053 -0.20090303 ... -0.5092494  -0.7242769
    0.0037366 ]
  [ 0.42569858  0.49202585 -0.1921086  ... -0.52926743 -0.73894566
    0.00663666]]

 [[ 0.49737298  0.5550515  -0.13341121 ... -0.5284789  -0.61509997
   -0.04155324]
  [ 0.5157517   0.5635947  -0.09799525 ... -0.5260042  -0.62772596
   -0.03477238]
  [ 0.53440964  0.5700865  -0.05099516 ... -0.5282975  -0.63185966
   -0.02259888]
  ...
  [ 0.39933348  0.4695823  -0.19357261 ... -0.49450147 -0.70106435
   -0.01530197]
  [ 0.41170758  0.47419053 -0.20090303 ... -0.5092494  -0.7242769
    0.0037366 ]
  [ 0.42569858  0.49202585 -0.1921086  ... -0.52926743 -0.73894566
    0.00

In [30]:
sess = tf.InteractiveSession()


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [6]:
def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(q, k, v, None)
    print ('Attention weights are:')
    print (temp_attn.eval())
    print ('Output is:')
    print (temp_out.eval())

    
temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# This query aligns equally with the first and second key, 
# so their values get averaged.
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
print_out(temp_q, temp_k, temp_v)




Attention weights are:


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [7]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
print(out.shape, attn.shape)

sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512)))

NameError: name 'MultiHeadAttention' is not defined

models

In [29]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, target_vocab_size=8000)

temp_input = tf.random.uniform((64, 62))
temp_target = tf.random.uniform((64, 26))

fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)


TensorShape([Dimension(64), Dimension(26), Dimension(8000)])

### Optimizer

In [37]:
warmup_steps = 4000
initial_lr = 0.0003

def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    '''Noam scheme learning rate decay
    init_lr: initial learning rate. scalar.
    global_step: scalar.
    warmup_steps: scalar. During warmup_steps, learning rate increases
        until it reaches init_lr.
    '''
    step = tf.cast(global_step + 1, dtype=tf.float32)
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)


global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# train_op = optimizer.minimize(loss, global_step=global_step)

In [31]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """
    Use the Adam optimizer with a custom learning rate scheduler according to the formula
    """
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


AttributeError: module 'tensorflow._api.v1.keras.optimizers' has no attribute 'schedules'

### Losses and Metrics

In [43]:
loss_object = tf.keras.losses.sparse_categorical_crossentropy(from_logits=True)


TypeError: sparse_categorical_crossentropy() missing 2 required positional arguments: 'y_true' and 'y_pred'

In [41]:
def loss_function(real, pred):
    """
    Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')


In [44]:
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


In [25]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, target_vocab_size=8000)

output, attn = sample_decoder(tf.random.uniform((64, 26)), 
                              enc_output=sample_encoder_output, 
                              training=False, look_ahead_mask=None, 
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape, attn['decoder_layer2_block1'].shape


(TensorShape([Dimension(64), Dimension(26), Dimension(512)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(62)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(26)]))

In [21]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500)

sample_encoder_output = sample_encoder(tf.random.uniform((64, 62)), 
                                       training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)


(64, 62, 512)


In [18]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output, 
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)


TensorShape([Dimension(64), Dimension(50), Dimension(512)])

In [14]:
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 12:59:35.651068 140526081922880 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [9]:
config.SEQ_LEN

512